# Identificación de Retinopatía Diabética con Arquitectura de Aswin Shriram Thiagarajan et al

Paper: Aswin Shriram Thiagarajan et al., / Journal of Computer Science 2020, 16 (3): 305.313 

__DOI: 10.3844/jcssp.2020.305.313__

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths

from tesis_lib.nn.thiagarajan import Thiagarajan
from tesis_lib.io.hdf5datasetwriter import HDF5DatasetWriter
from tesis_lib.io.hdf5datasetgenerator import HDF5DatasetGenerator
from tesis_lib.datasets.rosenbrock_loader import RosenbrockLoader
from tesis_lib.callbacks.trainingmonitor import TrainingMonitor
from tesis_lib.preprocessing.imagetoarrayprocessor import ImageToArrayPreprocessor
from tesis_lib.preprocessing.aspectawareprocessor import AspectAwareProcessor
from tesis_lib.preprocessing.patchpreprocessor import PatchPreprocessor
from tesis_lib.preprocessing.meanpreprocessor import MeanPreprocessor
from tesis_lib.preprocessing.simplepreprocessor import SimpleProcessor

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import numpy as np
import progressbar
import json
import cv2
import os

2023-01-25 14:59:18.663699: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [6]:
from datetime import datetime

DATASET_PATH = './DB'

MODEL_OUT_PATH = f'./models/thiagarajan_{datetime.today().strftime("%d_%m_%Y_%H_%M_%S")}.h5'
BATCH_SIZE = 64
EPOCHS = 100
IM_SIZE = 224
NUM_CLASSES = 2

In [ ]:
DATASET_HDF5_PATH = os.path.sep.join([DATASET_PATH, 'hdf5'])
if os.path.exists(DATASET_HDF5_PATH):
    !rm -r {DATASET_HDF5_PATH}

os.mkdir(DATASET_HDF5_PATH)

In [ ]:
aap = AspectAwareProcessor(IM_SIZE,IM_SIZE)
# iap = ImageToArrayPreprocessor()
(R,G,B) = ([],[],[])

path = os.path.sep.join([DATASET_PATH, "Training"])
class_paths = [os.path.sep.join([path, im_class]) for im_class in os.listdir(path)]

imagePaths = []
[imagePaths.extend(paths.list_images(cp)) for cp in class_paths]
labels = [pt.split(os.path.sep)[-2] for pt in imagePaths]

le = LabelEncoder()
labels = le.fit_transform(labels)

(trainPaths, valPaths,trainLabels,valLabels) = train_test_split(
  imagePaths,
  labels,
  train_size=450,
  test_size=100,
  stratify=labels, 
  random_state = 42)

assert trainLabels.shape[0] == len(trainPaths)
assert valLabels.shape[0] == len(valPaths)

In [ ]:
path = os.path.sep.join([DATASET_PATH, "Test"])
class_paths = [os.path.sep.join([path, im_class]) for im_class in os.listdir(path)]

imagePaths = []
[imagePaths.extend(paths.list_images(cp)) for cp in class_paths]
labels = [pt.split(os.path.sep)[-2] for pt in imagePaths]

le = LabelEncoder()
labels = le.fit_transform(labels)

(_, testPaths,_,testLabels) = train_test_split(
  imagePaths,
  labels,
  train_size=450,
  test_size=100,
  stratify=labels, 
  random_state = 42)

assert testLabels.shape[0] == len(testPaths)

In [7]:
DATA_PATHS = [
    ('train', trainPaths, trainLabels, f'./DB/hdf5/Training.hdf5'),
    ('val', valPaths, valLabels, f'./DB/hdf5/Validation.hdf5'),
    ('test', testPaths, testLabels, f'./DB/hdf5/Testing.hdf5'),
]

for (dType, imagePaths, labels, output) in DATA_PATHS:
  if os.path.exists(output):
    os.remove(output)
  writer = HDF5DatasetWriter((len(imagePaths), IM_SIZE,IM_SIZE,3), output)

  widgets = [
      f"Building {dType} Set: ",
      progressbar.Percentage(),
      " ",
      progressbar.Bar(),
      " ",
      progressbar.ETA()
  ]

  pbar = progressbar.ProgressBar(
      maxval=len(imagePaths),
      widgets=widgets
      ).start()

  for (i, (path,label)) in enumerate(zip(imagePaths, labels)):
      image = cv2.imread(path)
      try:
        image = aap.preprocess(image)
      except Exception:
        display(f"[WARNING] Skipped {path.split('/')[-1]}")
        continue

      if dType == "train":
        (b,g,r) = cv2.mean(image)[:3]
        R.append(r)
        G.append(g)
        B.append(b)
      
      writer.add([image], [label])
      pbar.update(i)

  pbar.finish()
  writer.close()

Building train Set: 100% |######################################| Time: 0:00:31
Building val Set: 100% |########################################| Time: 0:00:09
Building test Set: 100% |#######################################| Time: 0:00:11


In [8]:
print("[INFO] serializing means...")
D = {
    "R": np.mean(R),
    "G": np.mean(G),
    "B": np.mean(B)
}
with open('./DB/hdf5/diat_ret.json', "w") as f:
    f.write(json.dumps(D))

[INFO] serializing means...


In [9]:
aug = ImageDataGenerator(
    rotation_range=25, 
    shear_range=0.20, 
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    horizontal_flip=True, 
    fill_mode='nearest')

In [7]:
means = json.loads(open('./DB/hdf5/diat_ret.json').read())

sp = SimpleProcessor(IM_SIZE, IM_SIZE)
pp = PatchPreprocessor(IM_SIZE,IM_SIZE)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [11]:
trainGen = HDF5DatasetGenerator(
    'DB/hdf5/Training.hdf5',
    batchSize=BATCH_SIZE,
    aug=aug,
    preprocessors=[pp, mp, iap],
    classes=NUM_CLASSES
)

valGen = HDF5DatasetGenerator(
    'DB/hdf5/Validation.hdf5',
    batchSize=BATCH_SIZE,
    aug=aug,
    preprocessors=[sp, mp, iap],
    classes=NUM_CLASSES
)

In [12]:
print("[INFO] Compiling model ...")

opt = Adam(lr=1e-3)

model = Thiagarajan.build(
    width= IM_SIZE,
    height= IM_SIZE,
    depth= 3,
    classes= NUM_CLASSES
)

model.compile(
    loss="binary_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

monitor_path = os.path.sep.join([
    './output/',
    f"{os.getpid()}.jpg"
])

callbacks = [TrainingMonitor(monitor_path)]

model.summary()

[INFO] Compiling model ...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    

2023-01-25 14:11:49.305238: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-25 14:11:49.305919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-25 14:11:49.317037: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 Laptop GPU computeCapability: 8.6
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 178.84GiB/s
2023-01-25 14:11:49.317084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-25 14:11:49.318587: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-01-25 14:11:49.318710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcubl

In [13]:
print(f"[INFO] training model ...")

model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // BATCH_SIZE,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // BATCH_SIZE,
    epochs= EPOCHS,
    max_queue_size=10,
    callbacks=callbacks,
    verbose=1
)

/home/newton/miniconda3/envs/tesisenv/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


[INFO] training model ...


2023-01-25 14:15:48.247813: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-01-25 14:15:48.248073: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3110400000 Hz


Epoch 1/100


2023-01-25 14:15:48.792621: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-01-25 14:17:05.374504: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


7/7 [==============================] - 381s 534ms/step - loss: 0.6790 - accuracy: 0.8021 - val_loss: 0.6287 - val_accuracy: 0.8125
Epoch 2/100
7/7 [==============================] - 3s 527ms/step - loss: 0.6064 - accuracy: 0.8280 - val_loss: 0.5629 - val_accuracy: 0.8125
Epoch 3/100
7/7 [==============================] - 3s 486ms/step - loss: 0.5499 - accuracy: 0.8123 - val_loss: 0.5105 - val_accuracy: 0.8125
Epoch 4/100
7/7 [==============================] - 3s 506ms/step - loss: 0.4900 - accuracy: 0.8296 - val_loss: 0.4854 - val_accuracy: 0.8125
Epoch 5/100
7/7 [==============================] - 3s 518ms/step - loss: 0.4619 - accuracy: 0.8307 - val_loss: 0.4837 - val_accuracy: 0.8125
Epoch 6/100
7/7 [==============================] - 3s 468ms/step - loss: 0.4873 - accuracy: 0.8114 - val_loss: 0.4881 - val_accuracy: 0.8125
Epoch 7/100
7/7 [==============================] - 3s 488ms/step - loss: 0.5531 - accuracy: 0.7748 - val_loss: 0.4884 - val_accuracy: 0.8125
Epoch 8/100
7/7 [======

In [14]:
print("[INFO] serializing model ...")
model.save(MODEL_OUT_PATH, overwrite=True)

trainGen.close()
valGen.close()

[INFO] serializing model ...


In [23]:
model = load_model('models/thiagarajan_25_01_2023_14_10_55.h5')

In [24]:
testGen = HDF5DatasetGenerator(
    'DB/hdf5/Testing.hdf5',
    batchSize=BATCH_SIZE,
    preprocessors=[sp, mp, iap],
    classes=NUM_CLASSES
)

In [27]:
predictions = model.predict_generator(
    testGen.generator(),
    steps=testGen.numImages // 64, 
    max_queue_size=10)

/home/newton/miniconda3/envs/tesisenv/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2023-01-25 15:25:49.605231: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-01-25 15:25:49.684196: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-01-25 15:25:49.684238: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at conv_ops_fused_impl.h:697 : Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/activation/Relu (defined at tmp/ipykernel_15572/3304702617.py:1) ]] [Op:__inference_predict_function_2252]

Function call stack:
predict_function
